In [3]:
import numpy as np
import sympy as sp
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic
import math
from numpy.linalg import matrix_power

In [4]:
def classRepresentatives(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if tr(Q)==np.round(trace)]

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def classCFReps(trace): # gives continued fractions of a given trace
    bound = math.floor(trace**2-4)
    cfList = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            cfList+=cfReps(D)
        elif D%4 == 1:
            cfList+=cfReps(D)
        i = i + 1
    return [cf for cf in cfList if cfTrace(cf)==np.round(trace)]

def cfReps(D):
    fracReps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                fracReps.append(frac)
            else:
                continue
    return fracReps

def classCFReps2(trace): # gives continued fractions of a given trace
    bound = math.floor(trace**2-4)
    cfList = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            cfList+=cfReps2(D)
        elif D%4 == 1:
            cfList+=cfReps2(D)
        i = i + 1
    return [cf for cf in cfList if cfTrace(cf)==np.round(trace)]

def cfReps2(D):
    fracReps = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            minRep = min(permutationsOf(frac))
            if minRep not in fracReps and gcd(A,B,C)==1:
                fracReps.append(minRep)
            else:
                continue
    return fracReps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

def cfTrace(cf):
    return np.trace(cfToMat(cf))

def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False

def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))

def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [60]:
def contFracToLR(contFrac):
    if len(contFrac)%2==1:
        contFrac = contFrac + contFrac
    output = []
    for i in range(len(contFrac)):
        if i%2==1: #if original cont'd frac is even length, making last one R/alternating is same as this (hopefully)
            for j in range(contFrac[i]):
                output.append(1)
        else:
            for j in range(contFrac[i]):
                output.append(0)
    return output

def recover2(ls): #sends a whole list to a cont'd fraction (not clear whether this is needed)
    output = []
    i=0
    while i<len(ls):
        counter=1
        while(i<len(ls)-1 and ls[i] == ls[i+1]):
            i+=1
            counter+=1
        i+=1
        output.append(counter)
    if (len(output)%2!=0):
        output[len(output)-1]+=output[0]
        output = output[1:]
    return output

def indices(lrSeq): #positions of R's (R=1)
    return [i for i in range(len(lrSeq)) if lrSeq[i]==1]

def pairs(lrSeq): #pairs of R's on which to split
    output = []
    for i in indices(lrSeq):
        for j in indices(lrSeq):
            if i==j:
                continue
            else:
                output.append((i,j))
    return output

def split(contFrac): #splits a cont'd fraction on every possible pair of R's into 2 groups of R's and L's (in 0's and 1's)
    output = []
    converted = contFracToLR(contFrac)
    n = len(converted)
    splitPoints = pairs(converted)
    for i,j in splitPoints:
        temp = converted[i:]+converted[:i]
        temp1 = temp[1:(j-i)%n]
        temp2 = inv(temp[(j-i)%n+1:]) #inv(temp[(j-i)%n+1:])
        output.append((tuple(temp1),tuple(temp2)))
    return output

def printCon(lrSeq): #R = 1, L = 0
    a = ''
    for i in lrSeq:
        if i==1:
            a+= 'R'
        else:
            a+= 'L'
    return a

def printConLs(ls):
    printList = []
    for L in ls:
        printList.append(printCon(L))
    return printList

#def getDict(contFrac):
#    output = []
#    fracs = []
#    for t in range(1+2*e):
#        tr = trace-e+t
#        fracs += [ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] for Q in classRepresentatives(tr)]
#    for f in fracs:
#        output += split(f)
#    return output

#def cfListToDict(cfList):
#    outputDict = {}
#    for cf in cfList:
#        for obj in split(cf):
#            if obj[0] in outputDict:
#                outputDict[obj[0]].add(obj[1])
#            else:
#                outputDict[obj[0]] = {obj[1]}
#    return outputDict

def cfListToDicts(cfList):
    forwardDict = {}
    invDict = {}
    for cf in cfList:
        for obj in split(cf):
            if obj[0] in forwardDict:
                forwardDict[obj[0]].add(obj[1])
            else:
                forwardDict[obj[0]] = {obj[1]}
            if obj[1] in invDict:
                invDict[obj[1]].add(obj[0])
            else:
                invDict[obj[1]] = {obj[0]}
    return forwardDict,invDict

def inv(lrSeq): #inverts a list of 0's and 1's (R's and L's)
    return tuple((np.array(lrSeq[::-1])-1)**2)

def pairToFrac(s1,s2):
    out = list(s1)+[1]+list(inv(s2))+[1]
    return tuple(min(permutationsOf(recover2(out))))

#def getPants(cfList): #this clearly doesn't work
#    allPairs = cfListToDict(cfList)
#    output,fracs = [],[]
#    for s0,s1 in allPairs:
#        temp1,temp2 = set([tuple(a[1]) for a in allPairs if a[0]==s0]), set([tuple(a[1]) for a in allPairs if a[0]==s1])
#        s  = temp1.intersection(temp2)
#        for s2 in s:
#            s2 = list(s2)
#            output.append([s0,s1,s2])
#            fracs = [pairToFrac(s0,inv(s1)),pairToFrac(s1,inv(s2)),pairToFrac(s2,inv(s0))]
#    return fracs

def getPants(cfList):
    dictPair = cfListToDicts(cfList)
    forwardDict = dictPair[0]
    invDict = dictPair[1]
    pants = set()
    for s0 in forwardDict:
        for s1 in forwardDict[s0]:
            if s1 in forwardDict and s0 in invDict:
                for s2 in forwardDict[s1].intersection(invDict[s0]):
                    y0,y1,y2 = pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)
                    if y0==y1 or y1==y2 or y0==y2:
                        continue
                    pants.add(frozenset((y0,y1,y2)))
    return pants

def getPants2(cfList):
    dictPair = cfListToDicts(cfList)
    forwardDict = dictPair[0]
    invDict = dictPair[1]
    pants = []
    for s0 in forwardDict:
        for s1 in forwardDict[s0]:
            if s1 in forwardDict and s0 in invDict:
                for s2 in forwardDict[s1].intersection(invDict[s0]):
                    y0,y1,y2 = pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)
                    if y0==y1 or y1==y2 or y0==y2:
                        continue
                    pants.append((list((s0,s1,s2))))
    return pants    

def pantsInTraceRange(traceMin,traceMax): #output: triple of three geos with 0s
    cfList = []
    for trace in range(traceMin,traceMax+1):
        cfList += classCFReps(trace)
    return getPants(cfList)

def pantsInTraceRange2(traceMin,traceMax): #output: triple of theta sequences
    cfList = []
    for trace in range(traceMin,traceMax+1):
        cfList += classCFReps(trace)
    return getPants2(cfList)

def pantsInTraceRangeWithCuff(traceMin,traceMax,cuttingSequence):
    allPants = pantsInTraceRange(traceMin,traceMax)
    pantsWithCuff = []
    for pants in allPants:
        if min(permutationsOf(cuttingSequence)) in pants:
            pantsWithCuff.append(pants)
    return pantsWithCuff

def pantsInTraceRangeWithCuff2(traceMin,traceMax,cuttingSequence):
    allPants = pantsInTraceRange2(traceMin,traceMax)
    pantsWithCuff = []
    for pants in allPants:
        s0 = pants[0]
        s1 = pants[1]
        s2 = pants[2]
        y0,y1,y2 = pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)
        if min(permutationsOf(cuttingSequence)) == y0 or min(permutationsOf(cuttingSequence)) == y1 or min(permutationsOf(cuttingSequence)) == y2:
            pantsWithCuff.append(pants)
    return pantsWithCuff

def pantsWithCuffGivenList(pants_list,cuttingSequence):
    pantsWithCuff = []
    for pants in pants_list:
        s0 = pants[0]
        s1 = pants[1]
        s2 = pants[2]
        y0,y1,y2 = pairToFrac(s0,s1),pairToFrac(s1,s2),pairToFrac(s2,s0)
        if min(permutationsOf(cuttingSequence)) == y0 or min(permutationsOf(cuttingSequence)) == y1 or min(permutationsOf(cuttingSequence)) == y2:
            pantsWithCuff.append(pants)
    return pantsWithCuff

def boundaryCountForOrientedCuffs(traceMin,traceMax,cuttingSequencePositive): #i sort of screwed up orientation here
                                                                                #since surface is on the left and therefore
                                                                                #we prefer counterclockwise...
    cuttingSequenceNegative = cuttingSequencePositive[::-1]
    positiveOrientationPants = pantsInTraceRangeAndWithGivenCuff(traceMin,traceMax,cuttingSequencePositive)
    negativeOrientationPants = pantsInTraceRangeAndWithGivenCuff(traceMin,traceMax,cuttingSequenceNegative)
    return (len(positiveOrientationPants) - len(negativeOrientationPants))
            

def cfToMat(cf):
    mat = np.identity(2)
    length = 0
    for x in cf:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)


def pantsToGeodesics(pList):
    positives = list()
    for p in pList:
        for geo in p:
            minGeo = min(permutationsOf(geo))
            invGeo = min(permutationsOf(minGeo[::-1]))
            if minGeo in positives or invGeo in positives:
                continue
            else:
                positives.append(minGeo)
    return positives
                
def geosToEyes(geos):
    eyes = list()
    for geo in geos:
        for i in range(0,len(geo),2):
            a = list(geo[i:])+list(geo[:i])
            a = a[1:]
            a[0]-=1
            a[len(a)-1]-=1
            eyes.append((geo,tuple(a)))
    return eyes

def eyesToCusps(eList):
    output = list()
    for e in eList:
        eye = e[1]
        invEye = eye[::-1]
        if eye in output or invEye in output:
            continue
        else:
            output.append(eye)
    return output
                
def pantsAndEyesToMatrix(pantList, eyeList, posGeoList, posCuspList):
    
    geoIndexDict = {}
    cuspIndexDict = {}
    
    basisList = posGeoList + posCuspList
    
    for i in range(len(basisList)):
        if i < len(posGeoList):
            geoIndexDict[posGeoList[i]] = i 
        else:
            cuspIndexDict[posCuspList[i-len(posGeoList)]] = i
        
    colList = []
    for pants in pantList:
        col = np.zeros(len(basisList))
        for geo in pants:
            if geo in geoIndexDict:
                col[geoIndexDict[geo]] = 1
            else:
                col[geoIndexDict[min(permutationsOf(geo[::-1]))]] = -1
        colList.append(col)
    
    for eye in eyeList:
        col = np.zeros(len(basisList))
        
        if eye[0] in geoIndexDict:
            col[geoIndexDict[eye[0]]] = 1
        else:
            col[geoIndexDict[min(permutationsOf(eye[0][::-1]))]] = -1
        
        if eye[1] in cuspIndexDict:
            col[cuspIndexDict[eye[1]]] = 1
        else:
            col[cuspIndexDict[eye[1][::-1]]] = -1
        
        colList.append(col)
    
    return np.column_stack(colList)



In [36]:
def pantsInPlane(pants): #y0 is on top, y2 on left, y1 in default position (reduced position +1)
    circles = []
    s0,s1,s2 = pants
    y2 = pairToFrac2(s0,s1)
    y0 = pairToFrac2(s1,s2)
    y1 = pairToFrac2(s2,s0)
    B = lambda x : 1/(1-x)
    
    alpha = ntheory.continued_fraction_reduce([y0])
    alpha_conj = -1/(ntheory.continued_fraction_reduce([y0[::-1]]))
    x_min = B(alpha_conj+1)
    x_max = B(alpha+1)
    r = (x_max-x_min)/2
    r = r.evalf()
    x = (x_min+x_max)/2
    x = x.evalf()
    circles.append((r,x))
    
    alpha = ntheory.continued_fraction_reduce([y1])
    alpha_conj = -1/(ntheory.continued_fraction_reduce([y1[::-1]]))
    x_min = (alpha_conj+1)
    x_max = (alpha+1)
    r = (x_max-x_min)/2
    r = r.evalf()
    x = (x_min+x_max)/2
    x = x.evalf()
    circles.append((r,x))
    
    alpha = ntheory.continued_fraction_reduce([y2])
    alpha_conj = -1/(ntheory.continued_fraction_reduce([y2[::-1]]))
    x_min = B(B(alpha_conj+1))
    x_max = B(B(alpha+1))
    r = (x_max-x_min)/2
    r = r.evalf()
    x = (x_min+x_max)/2
    x = x.evalf()
    circles.append((r,x))
    return circles

In [7]:
def getInv(circ):
    return ((circ[1]**2-circ[0]**2)/circ[0],1/circ[0],circ[1]/circ[0])

In [8]:
def getDist(circ1,circ2):
    circ1,circ2 = getInv(circ1), getInv(circ2)
    t1,u1,v1 = circ1
    t2,u2,v2 = circ2
    a = (t1*u2/2+t2*u1/2-v1*v2)
    return a

In [9]:
def act(M,x):
    return (M[0,0]*x+M[0,1])/(M[1,0]*x+M[1,1])

In [10]:
def rotl(x, k):
    return x[k:] + x[:k]

In [11]:
def permutationsOfGen(sequence):
    for n in range(0, len(sequence), 1 if isOdd(sequence) else 2):
        yield rotl(sequence, n)

In [12]:
def minPermutation(sequence):
    return min(permutationsOfGen(sequence))

In [13]:
def isOdd(sequence):
    return len(sequence) % 2

In [14]:
def recover2(ls):
    output = []
    i = 0
    while i < len(ls):
        counter = 1
        while (i < len(ls) - 1 and ls[i] == ls[i + 1]):
            i += 1
            counter += 1
        i += 1
        output.append(counter)
    if isOdd(output):  
        output = [0]+output
    return output

In [15]:
def pairToFrac2(s1,s2):
    return recover2(list(s1) + [1] + list(inv(s2)) + [1])

In [16]:
def pantsShear(pants1, pants2, y): # y is given in minimal form, without any 0's 
    
    s0,s1,s2 = pants1
    pants1Arr = []
    
    pants1Arr.append(pairToFrac(s1,s2))
    pants1Arr.append(pairToFrac(s2,s0))
    pants1Arr.append(pairToFrac(s0,s1))
    
    #pair to frac here sends extra R's at the front to the end, so it makes sense to compare to y later.
    
    i = -1
    for l in range(3):
        if pants1Arr[l] == y:
            i = l
    
    pants1_new = [pairToFrac2(s1,s2),pairToFrac2(s2,s0),pairToFrac2(s0,s1)]
    pants1_new = pants1_new[i:] + pants1_new[:i]
    pants1  = pants1[i:] + pants1[:i]
    
    #the above puts the cuff that matches to y into the first slot of the array
    #example: if i = 1, then pants1_new = [gamma1,gamma2,gamma0], and pants1 = [s1,s2,s0]
    
    s0,s1,s2 = pants2
    
    pants2Arr = []
    
    pants2Arr.append(pairToFrac(s1,s2))
    pants2Arr.append(pairToFrac(s2,s0))
    pants2Arr.append(pairToFrac(s0,s1))
    
    for l in range(3):
        if minPermutation(pants2Arr[l][::-1]) == y:
            i = l
    
    pants2_new = [pairToFrac2(s1,s2),pairToFrac2(s2,s0),pairToFrac2(s0,s1)]
    pants2_new = pants2_new[i:] + pants2_new[:i]
    pants2 = pants2[i:] + pants2[:i]
    
    #the above puts the cuff that matches to the inverse of y into the first slot of the array
    
    gamma_1 = pants1_new[0]
    gamma_2 = pants2_new[0]
    
    #the above are the two geodesics we want to match
    A_1 = np.eye(2)
    A_2 = np.eye(2)
    
    if gamma_1[0]==0:
        A_1 = np.linalg.inv(np.array([[0,1],[1,-gamma_1[1]]]) @ np.array([[0,1],[1,0]]))
        gamma_1[-1]+=gamma_1[1]
        gamma_1 = gamma_1[2:]
    
    #the above occurs when the start of the geo cutting sequence was an R and we then put a 0 in the cont frac
    #to represent L^0
    #we then store A_1 as the matrix that acts on the continued fraction by concatenating gamma_1[1] to the front
    #next, we add gamma_1[1] to the last entry as that is how 0 works for continued fractions, and then take away entries
        
    
    if gamma_2[0]==0:
        A_2 = np.array([[0,1],[1,-gamma_2[1]]]) @ np.array([[0,1],[1,0]])
        gamma_2[-1]+=gamma_2[1]
        gamma_2 = gamma_2[2:]
        
    #the above stored matrix action on the cont frac is: delete the 0, then delete gamma_2[1] (really, rotate by 2 slots)
    
    gamma_2 = gamma_2[::-1]
    l = len(gamma_1)
    #reverse gamma_2 so we can compare it to gamma_1
    k = -1
    for i in range(0,l,2):
        if (gamma_2[i:]+gamma_2[:i]) == gamma_1:
            k = i
    #the above marks which even permutation we need to take to match back to gamma_1
    
    mat = np.array([[0,1],[-1,0]])
    #matrixList = []
    #matrixList.append(mat)
    for i in range(k):
        mat = np.matmul(np.array([[0,1],[1,-gamma_2[i]]]), mat)
        #matrixList.append(mat)
    #matrixList = matrixList[::-1]
        
    #the original matrix is -1/z action that reverses the order of the cont frac of conj(z)
    #the other multiplications take away entries now from the back of gamma_2
        
    B = np.array([[0,1],[-1,1]])
    
    #the above matrix permutes 0,1,infty cyclically. represented by: 1/(1-z)
    
    T = np.array([[1,1],[0,1]])
    
    #the above adds 1 to the cont frac
    
    T_inv = np.array([[1,-1],[0,1]])
    
    #the above subtracts 1 from the cont frac
    
    #matrixList.append(A_2)
    #matrixList.append(T_inv)
    #matrixList.append(matrix_power(B,2))
    #matrixList2 = [B] + [T] + [A_1] + matrixList
    
    M = ((((((B @ T) @ A_1) @ mat) @ A_2) @ T_inv) @ matrix_power(B, 2))
    
    #z may be represented by y^-1, but
        #we want M to act on the visual point: -1/z = 1/(1- (z + 1)) as we shifted things around so that the 
            #point on pants2 to consider is B(z+1) = -1/z
        #also want z to be sent to the conjugate of the visual point 
    #since the input is B(z+1), we have B*T*A_1*mat*A_2*T_inv acting on z+1
    #next subtract 1 and obtain z acted on by B*T*A_1*mat*A_2
    #if we had a 0 in gamma_2, then use A_2 to rotate away the first two entries. otherwise it is identity
    #use mat to send this to -1/z', the reverse order of the cont frac for conj(z') where z' is the rotation from before
    #if we had a 0 in gamma_1, then use A_1 to replace first entry by gamma_1[1]
    #add back 1
    #apply B again to go back to original place
    
    pants2_in_plane = pantsInPlane(pants2)
    min_max = [(x-r,x+r) for (r,x) in pants2_in_plane]
    pants2_in_plane = [(act(M, mini),act(M,maxi)) for (mini,maxi) in min_max]
    #for (mini,maxi) in min_max:
    #    for matrix in matrixList2:
    #        print((act(matrix,mini), act(matrix,maxi)))
    pants2_in_plane = [((x_max-x_min)/2,(x_max+x_min)/2) for (x_min,x_max) in pants2_in_plane]
    pants1_in_plane  = pantsInPlane(pants1)
    return pants1_in_plane+ pants2_in_plane

In [17]:
def getOrtho(circ1,circ2):
    r1,x1 = circ1 
    r2,x2 = circ2
    c1 = x1**2-r1**2
    c2 = x2**2-r2**2
    h = (c1-c2)/(2*(x2-x1))
    x3 = -h
    r3 = math.sqrt(h**2+2*x1*h+c1)
    return (r3, x3)

In [18]:
def getShear(pants1,pants2, gamma):
    circlesList = pantsShear(pants1,pants2,gamma)
    circ1 = getOrtho((abs(circlesList[2][0]),circlesList[2][1]),(abs(circlesList[3][0]),circlesList[3][1]))
    circ2 = getOrtho((abs(circlesList[3][0]),circlesList[3][1]),(abs(circlesList[4][0]),circlesList[4][1]))
    circ3 = getOrtho((abs(circlesList[1][0]),circlesList[1][1]),(abs(circlesList[3][0]),circlesList[3][1]))
    circ4 = getOrtho((abs(circlesList[3][0]),circlesList[3][1]),(abs(circlesList[5][0]),circlesList[5][1]))
    circ3 = (-circ3[0],circ3[1])
    circ1 = (-circ1[0],circ1[1])
    return (getDist(circ1,circ2),getDist(circ3,circ4))

In [61]:
def bruteShearsOnTraceRange(tmin, tmax):
    shear_list = []
    pre_pants_list = list(pantsInTraceRange(tmin,tmax))
    pos_geo_list = pantsToGeodesics(pre_pants_list) #remember to import pantsToGeodesics. remember the geos here are minimal
    pants_list = list(pantsInTraceRange2(tmin,tmax))
    #also need to fix the rep of these pants to include 0s... the functions called here get rid of 0s
    for geo in pos_geo_list:
        inv_geo = geo[::-1]
        list_of_pants_with_cuff = []
        list_of_pants_with_inv_cuff = []
        length = len(geo)
        for i in range(0,length,2):
            list_of_pants_with_cuff += pantsInTraceRangeWithCuff2(tmin, tmax, geo[i:] + geo[:i])
            list_of_pants_with_inv_cuff += pantsInTraceRangeWithCuff2(tmin, tmax, inv_geo[i:] + inv_geo[:i])
        for pants in list_of_pants_with_cuff:
            for inv_pants in list_of_pants_with_inv_cuff:
                print(pants, inv_pants, np.trace(cfToMat(geo)))
                shear = getShear(pants,inv_pants,geo)
                print(shear)
                shear_list.append(shear) #still need to fix rep of pants
    return shear_list

In [62]:
def basisPantsGivenTraceRange(tmin, tmax):
    shear_list = []
    pre_pants_list = list(pantsInTraceRange(tmin,tmax)) #pants list as geos without 0s
    pos_geo_list = pantsToGeodesics(pre_pants_list) #recall the geos here are minimal, no 0s
    pants_list = list(pantsInTraceRange2(tmin,tmax)) #pants list as three theta sequences
    basis_pants = []
    for geo in pos_geo_list:
        pantsWithCuff = pantsWithCuffGivenList(pants_list, geo) #this can be optimized to just find one pants
        pants = pantsWithCuff[0]
        basis_pants.append(pants)
        print(geo,pants)
    return basis_pants

In [63]:
basisPantsGivenTraceRange(10,30)

(1, 1, 1, 7) [(0, 0), (0, 1, 1), (1, 0, 0, 0, 0)]
(2, 1, 3, 2) [(0, 0), (0, 1, 1, 1), (0, 0, 0, 0, 0, 0)]
(3, 5) [(0, 0), (0, 1, 1), (1, 0, 0, 0)]
(1, 2, 4, 1) [(0,), (1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0)]
(2, 4) [(0, 0), (1, 0, 0), (0, 0)]
(1, 1, 8, 1) [(0,), (1, 1, 1, 1, 1, 1, 1, 1), (1, 0, 0)]
(1, 13) [(0, 0), (1, 1, 1, 1, 1, 1), (1, 0, 0, 0, 0, 0)]
(3, 4) [(0, 0), (1, 0, 0), (0, 0, 0)]
(1, 1, 1, 1, 1, 2) [(0,), (1, 0, 1, 0), (0, 0, 0, 0, 0)]
(1, 9) [(0, 0), (0, 1, 1), (0, 0, 0, 0, 0)]
(1, 2, 5, 1) [(0,), (1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0)]
(4, 5) [(0, 0), (0, 1, 1), (1, 0, 0, 0, 0)]
(2, 2, 3, 1) [(0, 0), (1, 1, 1, 0), (0, 0, 0, 0, 0, 0)]
(2, 13) [(0,), (0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0)]
(8, 2) [(), (1, 0, 1, 1, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0)]
(1, 14) [(0,), (1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0)]
(1, 1, 2, 2) [(0, 0), (1, 0), (0, 0, 0, 0, 0, 0)]
(1, 2, 1, 6) [(0, 0), (1, 1, 1, 1), (1, 0, 0, 1)]
(3, 3) [(0,), (1, 0, 1, 0), (0, 0, 0)]
(1, 2, 2, 1) [(0, 0), (0, 1), (0, 0

[[(0, 0), (0, 1, 1), (1, 0, 0, 0, 0)],
 [(0, 0), (0, 1, 1, 1), (0, 0, 0, 0, 0, 0)],
 [(0, 0), (0, 1, 1), (1, 0, 0, 0)],
 [(0,), (1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0)],
 [(0, 0), (1, 0, 0), (0, 0)],
 [(0,), (1, 1, 1, 1, 1, 1, 1, 1), (1, 0, 0)],
 [(0, 0), (1, 1, 1, 1, 1, 1), (1, 0, 0, 0, 0, 0)],
 [(0, 0), (1, 0, 0), (0, 0, 0)],
 [(0,), (1, 0, 1, 0), (0, 0, 0, 0, 0)],
 [(0, 0), (0, 1, 1), (0, 0, 0, 0, 0)],
 [(0,), (1, 1, 1, 1, 1, 0), (0, 0, 0, 0, 0, 0)],
 [(0, 0), (0, 1, 1), (1, 0, 0, 0, 0)],
 [(0, 0), (1, 1, 1, 0), (0, 0, 0, 0, 0, 0)],
 [(0,), (0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0)],
 [(), (1, 0, 1, 1, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0)],
 [(0,), (1, 1, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0)],
 [(0, 0), (1, 0), (0, 0, 0, 0, 0, 0)],
 [(0, 0), (1, 1, 1, 1), (1, 0, 0, 1)],
 [(0,), (1, 0, 1, 0), (0, 0, 0)],
 [(0, 0), (0, 1), (0, 0, 0, 0, 0, 0)],
 [(0,), (0, 1), (0, 1, 0)],
 [(0, 0), (0, 1, 1), (0, 1, 0)],
 [(0, 0), (0, 1, 1), (0, 0, 0, 0, 0, 0)],
 [(0, 0), (0, 1, 1), (0, 0, 0, 0, 0, 0, 0)],
 [(0, 

In [21]:
bruteShearsOnTraceRange(10,30)

[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 1, 1, 0, 1)] 10
(1.10216418024620, 1.10216418024620)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 1, 1, 1, 0, 0, 0)] 10
(1.00000000000000, 1.00000000000000)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 1, 0, 1, 1)] 10
(1.19207912135854, -1.58943882847805)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 0, 0, 0, 0, 0)] 10
(1.02296675412073, 1.02296675412073)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (0, 1, 1, 1, 1, 1)] 10
(1.58614105220650, -1.58614105220650)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 0, 1, 1, 0)] 10
(1.07763031587769, 1.07763031587769)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (0, 0, 1, 1)] 10
(1.57216231947285, -1.57216231947286)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (0, 0, 0, 0, 1, 1, 1)] 10
(1.83121458836896, -1.83121458836896)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 0, 0, 1, 0)] 10
(1.07861876819749, 1.07861876819749)
[(0,), (0, 1), (1, 0, 0)] [(0, 0), (1,), (1, 0, 1, 0, 0)] 10
(1.03590011911297, 1.03590011911297)
[

KeyboardInterrupt: 

In [125]:
def bruteShearsOnTraceRangeWithEpsilon(tmin, tmax, epsilon):
    s_min = np.cosh(1 - epsilon)
    s_max = np.cosh(1 + epsilon)
    shear_list = []
    pre_pants_list = list(pantsInTraceRange(tmin,tmax))
    pos_geo_list = pantsToGeodesics(pre_pants_list) #remember to import pantsToGeodesics. remember the geos here are minimal
    pants_list = list(pantsInTraceRange2(tmin,tmax))
    #also need to fix the rep of these pants to include 0s... the functions called here get rid of 0s
    for geo in pos_geo_list:
        inv_geo = geo[::-1]
        list_of_pants_with_cuff = []
        list_of_pants_with_inv_cuff = []
        length = len(geo)
        for i in range(0,length,2):
            list_of_pants_with_cuff += pantsInTraceRangeWithCuff2(tmin, tmax, geo[i:] + geo[:i])
            list_of_pants_with_inv_cuff += pantsInTraceRangeWithCuff2(tmin, tmax, inv_geo[i:] + inv_geo[:i])
        for pants in list_of_pants_with_cuff:
            for inv_pants in list_of_pants_with_inv_cuff:
                shear = getShear(pants,inv_pants,geo)
                if s_min < shear[0] < s_max or s_min < shear[1] < s_max: 
                    print(shear)
                    shear_list.append(shear) #still need to fix rep of pants
    return shear_list

In [130]:
bruteShearsOnTraceRangeWithEpsilon(10,30,.5)

(1.19207912135854, -1.58943882847805)
(1.58614105220650, -1.58614105220650)
(1.57216231947285, -1.57216231947286)
(1.83121458836896, -1.83121458836896)
(1.66554297974344, -1.66554297974344)
(1.73205080756887, -1.73205080756888)
(1.48999332201181, -1.48999332201181)
(1.65417638468990, -1.65417638468990)
(1.19207912135854, 1.58943882847805)
(1.63591872103348, -1.63591872103348)
(1.76427709961064, -1.76427709961064)
(1.71019462446017, -1.71019462446016)
(1.28509369232193, -1.28509369232193)
(1.19207912135854, -0.931740002900927)
(1.84210526315789, -1.84210526315789)
(1.61371915468513, -1.61371915468513)
(1.55094959495723, -1.55094959495723)
(1.31335731658081, -1.31335731658081)
(1.78190173228363, -1.78190173228363)
(1.19207912135854, nan)
(1.55094959495723, -1.55094959495723)
(1.49948772601859, -1.49948772601859)
(1.19207912135854, nan)
(1.36380600142198, -1.36380600142198)
(1.38157398892026, -1.38157398892026)
(1.36153516641322, -1.36153516641322)
(1.94693672558462, 1.94693672558461)
(1.

TypeError: Invalid NaN comparison

In [6]:
len(pantsInTraceRange(10,30))

1045

In [53]:
pantSet = pantsInTraceRange(10,30)

In [54]:
posGeoList = pantsToGeodesics(pantSet)

In [55]:
print(posGeoList)

[(1, 1, 1, 7), (2, 1, 3, 2), (3, 5), (1, 2, 4, 1), (2, 4), (1, 1, 8, 1), (1, 13), (3, 4), (1, 1, 1, 1, 1, 2), (1, 9), (1, 2, 5, 1), (4, 5), (2, 2, 3, 1), (2, 13), (8, 2), (1, 14), (1, 1, 2, 2), (1, 2, 1, 6), (3, 3), (1, 2, 2, 1), (1, 1, 1, 3), (1, 2, 1, 4), (2, 1, 2, 2), (7, 4), (1, 11), (1, 2, 3, 1), (9, 3), (1, 16), (10, 2), (1, 10), (3, 6), (1, 1, 2, 4), (1, 8), (2, 1, 3, 1), (1, 1, 1, 5), (1, 1, 5, 2), (1, 3, 1, 4), (1, 2, 3, 2), (1, 1, 1, 4), (2, 6), (1, 1, 3, 2), (6, 4), (1, 12), (1, 1, 1, 6), (2, 5), (2, 9), (2, 1, 5, 1), (1, 1, 2, 1), (1, 1, 3, 3), (1, 15), (4, 4), (7, 2), (1, 3, 3, 1), (1, 18), (11, 2), (1, 20), (3, 7), (14, 2), (1, 25), (8, 3), (1, 22), (12, 2), (1, 19), (5, 5), (1, 21), (1, 24), (1, 23), (1, 17)]


In [66]:
eyeList = geosToEyes(posGeoList)

In [67]:
posCuspList = eyesToCusps(eyeList)

In [38]:
posCuspList

[(1, 6, 0),
 (0, 2, 1),
 (0, 2, 8),
 (8, 1, 0),
 (0, 1, 1, 1, 4),
 (0, 1, 5, 1, 0),
 (14,),
 (0, 2, 1, 1, 2),
 (0, 1, 3, 1, 0),
 (2,)]

In [68]:
pantList = list(pantSet)

In [69]:
mat = pantsAndEyesToMatrix(pantList,eyeList,posGeoList,posCuspList)

In [43]:
np.sum(mat, axis=0)

array([3., 3., 3., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 2., 2., 2.,
       0., 2.])

In [87]:
from pulp import *
import numpy as np

EPS = 1e-3

""" Input """
A = mat
b = np.zeros(A.shape[0])

""" MIP """
# Variables
x = np.empty(A.shape[1], dtype=object)
for i in range(A.shape[1]):
    x[i] = LpVariable("x" + str(i), lowBound=None, upBound=None, cat='Integer')

# Problem
prob = LpProblem("prob", LpMinimize)

# Objective
prob += np.sum(x)

# Constraints
for row in range(A.shape[0]):
    prob += np.dot(A[row], x) == b[row]

prob += np.sum(x) >= EPS  # forbid zero-vector

print(prob)

# Solve
status = prob.solve()
print(LpStatus[status])
print([value(x_) for x_ in x])

prob:
MINIMIZE
1*x0 + 1*x1 + 1*x10 + 1*x100 + 1*x1000 + 1*x1001 + 1*x1002 + 1*x1003 + 1*x1004 + 1*x1005 + 1*x1006 + 1*x1007 + 1*x1008 + 1*x1009 + 1*x101 + 1*x1010 + 1*x1011 + 1*x1012 + 1*x1013 + 1*x1014 + 1*x1015 + 1*x1016 + 1*x1017 + 1*x1018 + 1*x1019 + 1*x102 + 1*x1020 + 1*x1021 + 1*x1022 + 1*x1023 + 1*x1024 + 1*x1025 + 1*x1026 + 1*x1027 + 1*x1028 + 1*x1029 + 1*x103 + 1*x1030 + 1*x1031 + 1*x1032 + 1*x1033 + 1*x1034 + 1*x1035 + 1*x1036 + 1*x1037 + 1*x1038 + 1*x1039 + 1*x104 + 1*x1040 + 1*x1041 + 1*x1042 + 1*x1043 + 1*x1044 + 1*x1045 + 1*x1046 + 1*x1047 + 1*x1048 + 1*x1049 + 1*x105 + 1*x1050 + 1*x1051 + 1*x1052 + 1*x1053 + 1*x1054 + 1*x1055 + 1*x1056 + 1*x1057 + 1*x1058 + 1*x1059 + 1*x106 + 1*x1060 + 1*x1061 + 1*x1062 + 1*x1063 + 1*x1064 + 1*x1065 + 1*x1066 + 1*x1067 + 1*x1068 + 1*x1069 + 1*x107 + 1*x1070 + 1*x1071 + 1*x1072 + 1*x1073 + 1*x1074 + 1*x1075 + 1*x1076 + 1*x1077 + 1*x1078 + 1*x1079 + 1*x108 + 1*x1080 + 1*x1081 + 1*x1082 + 1*x1083 + 1*x1084 + 1*x1085 + 1*x1086 + 1*x1087 + 1*

Undefined
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.3596059e-05, -0.00010344828, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.3596059e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [90]:
resultList = pantList+eyeList
for i in range(len(x)):
    if value(x[i]) != 0.0:
        print(i, x[i], x[i].cat, value(x[i]))
        print(resultList[i])
        

11 x11 Integer 9.3596059e-05
frozenset({(1, 1, 1, 4, 1, 5), (2, 3, 4, 3), (1, 2, 1, 2, 4, 1)})
12 x12 Integer -0.00010344828
frozenset({(1, 1, 2, 1, 7, 1), (4, 1, 4, 4), (1, 6, 1, 12)})
53 x53 Integer 9.3596059e-05
frozenset({(1, 2, 3, 8), (1, 1, 4, 1, 1, 4), (3, 2, 3, 4)})
196 x196 Integer 7.3891626e-05
frozenset({(2, 1, 2, 1, 5, 1), (1, 4, 10, 1), (1, 4, 1, 16)})
214 x214 Integer -5.4187192e-05
frozenset({(2, 3, 5, 2), (2, 44), (1, 1, 35, 1)})
238 x238 Integer 0.00010344828
frozenset({(1, 1, 1, 4, 2, 2), (1, 1, 1, 1, 1, 10), (2, 1, 8, 3)})
423 x423 Integer -9.8522167e-06
frozenset({(9, 11), (2, 1, 8, 3), (1, 3, 2, 10)})
570 x570 Integer 9.8522167e-06
frozenset({(2, 1, 2, 1, 5, 1), (1, 5, 1, 13), (2, 1, 7, 4)})
736 x736 Integer -8.3743842e-05
frozenset({(1, 1, 1, 4, 1, 5), (2, 1, 2, 1, 5, 1), (1, 1, 1, 1, 2, 7)})
888 x888 Integer 0.00010344828
frozenset({(1, 1, 3, 14), (1, 1, 1, 1, 10, 1), (1, 1, 2, 2, 1, 5)})
903 x903 Integer -5.4187192e-05
frozenset({(2, 46), (1, 1, 1, 1, 7, 2), (36

In [86]:
min(permutationsOf((0, 1, 1, 1, 2, 1, 1)[::-1]))

(0, 1, 1, 2, 1, 1, 1)